## DeepAlpha短周期因子系列研究：DNN

### 回测

- 策略思想：基于模型的预测结果进行选股，选择当日排名靠前的50只股票买入，卖出其他持有的股票。
- 调仓周期：日频，每日换仓
- 资金管理：每只股票最大资金占用20%
- 手续费：买入0.0003，卖出0.0013

In [6]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (16, 9)

In [ ]:
# 本代码由可视化策略环境自动生成 2022年6月2日 17:21
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m10_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    from sklearn.model_selection import train_test_split
    data = input_1.read()
    x_train, x_val, y_train, y_val = train_test_split(data["x"], data['y'], test_size=0.1)
    data_1 = DataSource.write_pickle({'x': x_train, 'y': y_train})
    data_2 = DataSource.write_pickle({'x': x_val, 'y': y_val})
    return Outputs(data_1=data_1, data_2=data_2, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m10_post_run_bigquant_run(outputs):
    return outputs

from tensorflow.keras.callbacks import EarlyStopping
m5_earlystop_bigquant_run=EarlyStopping(monitor='val_mse', min_delta=0.0001, patience=3)
# 用户的自定义层需要写到字典中，比如
# {
#   "MyLayer": MyLayer
# }
m5_custom_objects_bigquant_run = {
 
}

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m24_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    pred_label = input_1.read_pickle()
    df = input_2.read_df()
    df = pd.DataFrame({'pred_label':pred_label[:,0], 'instrument':df.instrument, 'date':df.date})
    df.sort_values(['date','pred_label'],inplace=True, ascending=[True,False])
    return Outputs(data_1=DataSource.write_df(df), data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m24_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m19_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 50
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.2
    context.options['hold_days'] = 5

# 回测引擎：每日数据处理函数，每天执行一次
def m19_handle_data_bigquant_run(context, data):
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]

    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.perf_tracker.position_tracker.positions.items()}
        
    # 2. 生成卖出订单：hold_days天之后才开始卖出；对持仓的股票，按机器学习算法预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        equities = {e.symbol: e for e, p in context.perf_tracker.position_tracker.positions.items()}
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(
                lambda x: x in equities and not context.has_unfinished_sell_order(equities[x]))])))
        # print('rank order for sell %s' % instruments)
        for instrument in instruments:
            context.order_target(context.symbol(instrument), 0)
            cash_for_sell -= positions[instrument]
            if cash_for_sell <= 0:
                break

    # 3. 生成买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    buy_cash_weights = context.stock_weights
    buy_instruments = list(ranker_prediction.instrument[:len(buy_cash_weights)])
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:
            context.order_value(context.symbol(instrument), cash)

# 回测引擎：准备数据，只执行一次
def m19_prepare_bigquant_run(context):
    pass


m1 = M.instruments.v2(
    start_date='2019-01-01',
    end_date='2021-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.advanced_auto_labeler.v2(
    instruments=m1.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/data_history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/big_expr.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -5) / shift(open, -1) - 1

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=True,
    cast_label_int=False
)

m12 = M.standardlize.v9(
    input_1=m2.data,
    standard_func='ZScoreNorm',
    columns_input='label'
)

m3 = M.input_features.v1(
    features="""close_0
open_0
high_0
low_0 
amount_0
turn_0 
return_0
 
close_1
open_1
high_1
low_1
return_1
amount_1
turn_1
 
close_2
open_2
high_2
low_2
amount_2
turn_2
return_2
 
close_3
open_3
high_3
low_3
amount_3
turn_3
return_3
 
close_4
open_4
high_4
low_4
amount_4
turn_4
return_4
 
mean(close_0, 5)
mean(low_0, 5)
mean(open_0, 5)
mean(high_0, 5)
mean(turn_0, 5)
mean(amount_0, 5)
mean(return_0, 5)
 
ts_max(close_0, 5)
ts_max(low_0, 5)
ts_max(open_0, 5)
ts_max(high_0, 5)
ts_max(turn_0, 5)
ts_max(amount_0, 5)
ts_max(return_0, 5)
 
ts_min(close_0, 5)
ts_min(low_0, 5)
ts_min(open_0, 5)
ts_min(high_0, 5)
ts_min(turn_0, 5)
ts_min(amount_0, 5)
ts_min(return_0, 5) 
 
std(close_0, 5)
std(low_0, 5)
std(open_0, 5)
std(high_0, 5)
std(turn_0, 5)
std(amount_0, 5)
std(return_0, 5)
 
ts_rank(close_0, 5)
ts_rank(low_0, 5)
ts_rank(open_0, 5)
ts_rank(high_0, 5)
ts_rank(turn_0, 5)
ts_rank(amount_0, 5)
ts_rank(return_0, 5)
 
decay_linear(close_0, 5)
decay_linear(low_0, 5)
decay_linear(open_0, 5)
decay_linear(high_0, 5)
decay_linear(turn_0, 5)
decay_linear(amount_0, 5)
decay_linear(return_0, 5)
 
correlation(volume_0, return_0, 5)
correlation(volume_0, high_0, 5)
correlation(volume_0, low_0, 5)
correlation(volume_0, close_0, 5)
correlation(volume_0, open_0, 5)
correlation(volume_0, turn_0, 5)
  
correlation(return_0, high_0, 5)
correlation(return_0, low_0, 5)
correlation(return_0, close_0, 5)
correlation(return_0, open_0, 5)
correlation(return_0, turn_0, 5)
 
correlation(high_0, low_0, 5)
correlation(high_0, close_0, 5)
correlation(high_0, open_0, 5)
correlation(high_0, turn_0, 5)
 
correlation(low_0, close_0, 5)
correlation(low_0, open_0, 5)
correlation(low_0, turn_0, 5)
 
correlation(close_0, open_0, 5)
correlation(close_0, turn_0, 5)

correlation(open_0, turn_0, 5)"""
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=10
)

m16 = M.derived_feature_extractor.v3(
    input_data=m15.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=True,
    remove_extra_columns=False
)

m28 = M.standardlize.v8(
    input_1=m16.data,
    input_2=m3.data,
    columns_input='[]'
)

m13 = M.fillnan.v1(
    input_data=m28.data,
    features=m3.data,
    fill_value='0.0'
)

m7 = M.join.v3(
    data1=m12.data,
    data2=m13.data,
    on='date,instrument',
    how='inner',
    sort=True
)

m30 = M.chinaa_stock_filter.v1(
    input_data=m7.data,
    index_constituent_cond=['全部'],
    board_cond=['全部'],
    industry_cond=['全部'],
    st_cond=['全部'],
    delist_cond=['全部'],
    output_left_data=False
)

m26 = M.dl_convert_to_bin.v2(
    input_data=m30.data,
    features=m3.data,
    window_size=1,
    feature_clip=3,
    flatten=True,
    window_along_col='instrument'
)

m10 = M.cached.v3(
    input_1=m26.data,
    run=m10_run_bigquant_run,
    post_run=m10_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2021-01-02'),
    end_date=T.live_run_param('trading_date', '2021-12-31'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=10
)

m18 = M.derived_feature_extractor.v3(
    input_data=m17.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=True,
    remove_extra_columns=False
)

m25 = M.standardlize.v8(
    input_1=m18.data,
    input_2=m3.data,
    columns_input='[]'
)

m14 = M.fillnan.v1(
    input_data=m25.data,
    features=m3.data,
    fill_value='0.0'
)

m31 = M.chinaa_stock_filter.v1(
    input_data=m14.data,
    index_constituent_cond=['全部'],
    board_cond=['全部'],
    industry_cond=['全部'],
    st_cond=['全部'],
    delist_cond=['全部'],
    output_left_data=False
)

m27 = M.dl_convert_to_bin.v2(
    input_data=m31.data,
    features=m3.data,
    window_size=1,
    feature_clip=3,
    flatten=True,
    window_along_col='instrument'
)

m6 = M.dl_layer_input.v1(
    shape='98',
    batch_shape='',
    dtype='float32',
    sparse=False,
    name=''
)

m29 = M.dl_layer_batchnormalization.v1(
    inputs=m6.data,
    axis=-1,
    momentum=0.99,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer='he_normal',
    gamma_initializer='he_normal',
    moving_mean_initializer='he_normal',
    moving_variance_initializer='he_normal',
    beta_regularizer='None',
    beta_regularizer_l1=0,
    beta_regularizer_l2=0,
    gamma_regularizer='None',
    gamma_regularizer_l1=0,
    gamma_regularizer_l2=0,
    beta_constraint='None',
    gamma_constraint='None',
    name=''
)

m8 = M.dl_layer_dense.v1(
    inputs=m29.data,
    units=256,
    activation='relu',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='Zeros',
    kernel_regularizer='None',
    kernel_regularizer_l1=0,
    kernel_regularizer_l2=0,
    bias_regularizer='None',
    bias_regularizer_l1=0,
    bias_regularizer_l2=0,
    activity_regularizer='None',
    activity_regularizer_l1=0,
    activity_regularizer_l2=0,
    kernel_constraint='None',
    bias_constraint='None',
    name=''
)

m21 = M.dl_layer_dropout.v1(
    inputs=m8.data,
    rate=0.1,
    noise_shape='',
    name=''
)

m20 = M.dl_layer_dense.v1(
    inputs=m21.data,
    units=128,
    activation='relu',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='Zeros',
    kernel_regularizer='None',
    kernel_regularizer_l1=0,
    kernel_regularizer_l2=0,
    bias_regularizer='None',
    bias_regularizer_l1=0,
    bias_regularizer_l2=0,
    activity_regularizer='None',
    activity_regularizer_l1=0,
    activity_regularizer_l2=0,
    kernel_constraint='None',
    bias_constraint='None',
    name=''
)

m22 = M.dl_layer_dropout.v1(
    inputs=m20.data,
    rate=0.1,
    noise_shape='',
    name=''
)

m23 = M.dl_layer_dense.v1(
    inputs=m22.data,
    units=1,
    activation='linear',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='Zeros',
    kernel_regularizer='None',
    kernel_regularizer_l1=0,
    kernel_regularizer_l2=0,
    bias_regularizer='None',
    bias_regularizer_l1=0,
    bias_regularizer_l2=0,
    activity_regularizer='None',
    activity_regularizer_l1=0,
    activity_regularizer_l2=0,
    kernel_constraint='None',
    bias_constraint='None',
    name=''
)

m4 = M.dl_model_init.v1(
    inputs=m6.data,
    outputs=m23.data
)

m5 = M.dl_model_train.v1(
    input_model=m4.data,
    training_data=m10.data_1,
    validation_data=m10.data_2,
    optimizer='Adam',
    loss='mean_squared_error',
    metrics='mse',
    batch_size=1024,
    epochs=10,
    earlystop=m5_earlystop_bigquant_run,
    custom_objects=m5_custom_objects_bigquant_run,
    n_gpus=0,
    verbose='2:每个epoch输出一行记录'
)

m11 = M.dl_model_predict.v1(
    trained_model=m5.data,
    input_data=m27.data,
    batch_size=1024,
    n_gpus=0,
    verbose='2:每个epoch输出一行记录'
)

m24 = M.cached.v3(
    input_1=m11.data,
    input_2=m18.data,
    run=m24_run_bigquant_run,
    post_run=m24_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m19 = M.trade.v4(
    instruments=m9.data,
    options_data=m24.data_1,
    start_date='',
    end_date='',
    initialize=m19_initialize_bigquant_run,
    handle_data=m19_handle_data_bigquant_run,
    prepare=m19_prepare_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.SHA'
)


[2022-06-02 17:19:23.275507] INFO: moduleinvoker: instruments.v2 开始运行..

[2022-06-02 17:19:23.290455] INFO: moduleinvoker: 命中缓存

[2022-06-02 17:19:23.293480] INFO: moduleinvoker: instruments.v2 运行完成[0.017958s].

[2022-06-02 17:19:23.311381] INFO: moduleinvoker: advanced_auto_labeler.v2 开始运行..

[2022-06-02 17:19:23.321694] INFO: moduleinvoker: 命中缓存

...

[2022-06-02 17:19:24.078603] INFO: moduleinvoker: general_feature_extractor.v7 运行完成[0.014282s].

[2022-06-02 17:19:24.121273] INFO: moduleinvoker: derived_feature_extractor.v3 开始运行..

[2022-06-02 17:19:24.134795] INFO: moduleinvoker: 命中缓存

[2022-06-02 17:19:24.137132] INFO: moduleinvoker: derived_feature_extractor.v3 运行完成[0.015876s].

[2022-06-02 17:19:24.156469] INFO: moduleinvoker: standardlize.v8 开始运行..

## Target分析

In [ ]:
no_normal_df  = m2.data.read()
no_normal_df.head()

In [ ]:
# label收益率
plt.figure(figsize=(18, 6))
plt.subplot(131)
plt.hist(no_normal_df["label"])
plt.title("所有数据")

plt.subplot(132)
plt.hist(no_normal_df[no_normal_df["instrument"] == "000001.SZA"]["label"])
plt.title("单票数据")

plt.subplot(133)
plt.hist(no_normal_df[no_normal_df["date"] == "2019-01-02"]["label"])
plt.title("单天数据")

plt.show()

In [ ]:
norm_df = m12.data.read()
norm_df.head()

In [ ]:
# label标准化
plt.figure(figsize=(18, 6))
plt.subplot(131)
plt.hist(norm_df["label"])
plt.title("所有数据")

plt.subplot(132)
plt.hist(norm_df[norm_df["instrument"] == "000001.SZA"]["label"])
plt.title("单票数据")

plt.subplot(133)
plt.hist(norm_df[norm_df["date"] == "2019-01-02"]["label"])
plt.title("单天数据")

plt.show()

## Feature分析

In [ ]:
no_norm_feature = m16.data.read()
no_norm_feature.head()
# std(close_0, 5)

In [ ]:
feature_name = "std(close_0, 5)"

In [ ]:
plt.figure(figsize=(18, 6))
plt.subplot(131)
plt.hist(no_norm_feature[feature_name])
plt.title("所有数据")

plt.subplot(132)
plt.hist(no_norm_feature[no_norm_feature["instrument"] == "000002.SZA"][feature_name])
plt.title("单票数据")

plt.subplot(133)
plt.hist(no_norm_feature[no_norm_feature["date"] == "2019-01-02"][feature_name])
plt.title("单天数据")

plt.show()

In [ ]:
norm_feature = m13.data.read()
norm_feature.head()

# std(close_0, 5)

In [ ]:
plt.figure(figsize=(18, 6))
plt.subplot(131)
plt.hist(norm_feature[feature_name])
plt.title("所有数据")

plt.subplot(132)
plt.hist(norm_feature[norm_feature["instrument"] == "000002.SZA"][feature_name])
plt.title("单票数据")

plt.subplot(133)
plt.hist(norm_feature[norm_feature["date"] == "2019-01-02"][feature_name])
plt.title("单天数据")

plt.show()